In [ ]:
%env INFLUXDB_V2_ORG=alki
%env INFLUXDB_V2_TOKEN=token
%env INFLUXDB_V2_URL=http://localhost:8086

import os

CLIENT_HOST = os.getenv("INFLUXDB_V2_URL")
CLIENT_ORG = os.getenv("INFLUXDB_V2_ORG")
CLIENT_TOKEN = os.getenv("INFLUXDB_V2_TOKEN")

In [ ]:
def build_flux_customer_filter(customer_names):
    filter_ = " |> filter(fn: (r) => "
    if isinstance(customer_names, str):
        filter_ += f'r["CUSTOMER"] == "{customer_names})"'
    elif isinstance(customer_names, list):
        filter_ += f'r["CUSTOMER"] == "{customer_names.pop(0)}" '
        for name in customer_names:
            filter_ += f'or r["CUSTOMER"] == "{name}" '
        filter_ += ")"
    return filter_


build_flux_customer_filter(["ARGALYS", "LES MIRACULEUX"])

In [ ]:
from influxdb_client import InfluxDBClient

with InfluxDBClient(CLIENT_HOST, CLIENT_TOKEN, org=CLIENT_ORG) as client:
    sql = (
        (
            'from(bucket: "technical-test")'
            " |> range(start: -8y)"
            ' |> filter(fn: (r) => r["_measurement"] == "customer_quantity")'
        )
        + build_flux_customer_filter(
            ["ARGALYS", "LES MIRACULEUX", "MINCI DELICE", "NUTRAVANCE"]
        )
        + (
            ' |> filter(fn: (r) => r["_field"] == "QUANTITY")'
            ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")'
            ' |> drop(columns: ["_start", "_stop", "_measurement"])'
        )
    )
    query_api = client.query_api()
    df = query_api.query_data_frame(sql)

df

In [ ]:
from torchmetrics import (
    MetricCollection,
    MeanSquaredError,
    MeanAbsoluteError,
    MultioutputWrapper,
)

metrics = MetricCollection(
    {
        "mean_squared_error": MeanSquaredError(),
        "mean_absolute_error": MeanAbsoluteError(),
    }
)